Author: Lauren M. Sanders

Date: April 2020

Treehouse Childhood Cancer Initiative

University of California, Santa Cruz 

In [1]:
library(limma)
library(edgeR)

In [2]:
sessionInfo()

R version 3.6.1 (2019-07-05)
Platform: x86_64-apple-darwin15.6.0 (64-bit)
Running under: macOS Mojave 10.14.6

Matrix products: default
BLAS:   /Library/Frameworks/R.framework/Versions/3.6/Resources/lib/libRblas.0.dylib
LAPACK: /Library/Frameworks/R.framework/Versions/3.6/Resources/lib/libRlapack.dylib

locale:
[1] en_US.UTF-8/en_US.UTF-8/en_US.UTF-8/C/en_US.UTF-8/en_US.UTF-8

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] edgeR_3.28.0 limma_3.42.0

loaded via a namespace (and not attached):
 [1] locfit_1.5-9.1  Rcpp_1.0.3      lattice_0.20-38 digest_0.6.23  
 [5] crayon_1.3.4    IRdisplay_0.7.0 grid_3.6.1      repr_1.0.2     
 [9] jsonlite_1.6    evaluate_0.14   pillar_1.4.3    rlang_0.4.2    
[13] uuid_0.1-2      IRkernel_1.1    tools_3.6.1     compiler_3.6.1 
[17] base64enc_0.1-3 htmltools_0.4.0 pbdZMQ_0.3-3   

In [3]:
# read in expression
phgg <- read.delim('files/pHGG.exp.log2tpm1.tsv',header=TRUE,row.names=1,stringsAsFactors=T,check.names=F) 
dim(phgg)

[1] 58581    78

In [6]:
# read in metadata
met = read.delim('files/pHGG.metadata.csv',header=TRUE,row.names=1,stringsAsFactors=T,check.names=F,sep=',')
dim(met)

[1] 78  3

In [5]:
# reorder expression
phgg <- phgg[rownames(met)]

In [7]:
y <- DGEList(phgg)

In [10]:
design <- model.matrix(~0 + factor(met$HistoneStatus))
colnames(design) <- c("Mut", "WT")

In [11]:
# voom normalize the data
v <- voom(y,design,plot = FALSE)

In [12]:
cont.matrix <- makeContrasts(k27m = Mut - WT, levels=design)

In [13]:
vfit <- lmFit(v, design)
vfit <- contrasts.fit(vfit, contrasts=cont.matrix)
# Apply empirical Bayes smoothing to the standard errors.
efit <- eBayes(vfit) 

In [ ]:
summary(decideTests(efit,p.value=0.1))

In [14]:
# each gene assigned an integer: 0=not DE, 1=genes up the tested condition, -1=genes down in the condition
allgenes <- as.matrix(decideTests(efit,p.value=0.1))

[1] 58581     1

In [16]:
# subset to all differentially expressed genes
de <- data.frame(allgenes[allgenes[,1] != 0,])  
de <- merge(de, efit$p.value, by="row.names")
colnames(de) <- c("Gene", "DEStatus", "pvalue")
dim(de)

[1] 1905    3

In [17]:
write.table(de, 'pHGG.limma.csv', quote=FALSE, sep=',', row.names=FALSE, col.names=TRUE)